In [6]:
import numpy as np
from scipy import stats
from operator import gt
import libpysal.api as ps
from esda.moran import Moran_Local
import mapclassify.api as mc
import libpysal
from giddy.api import Spatial_Markov, Markov
import giddy

In [2]:
f = libpysal.open(libpysal.examples.get_path("usjoin.csv"))
pci = np.array([f.by_col[str(y)] for y in range(1929,2010)])
pci = pci.transpose()
rpci = pci/(pci.mean(axis=0))
w = libpysal.open(libpysal.examples.get_path("states48.gal")).read()
w.transform = 'r'

In [3]:
y = rpci
y

array([[0.5250254 , 0.49080882, 0.49080203, ..., 0.86148267, 0.86252252,
        0.86746356],
       [0.97527938, 0.95588235, 0.93997352, ..., 0.90139505, 0.87897455,
        0.86216858],
       [0.50389434, 0.41911765, 0.4710823 , ..., 0.83675961, 0.83574199,
        0.84647177],
       ...,
       [0.74771419, 0.75      , 0.78002465, ..., 0.80172182, 0.82168155,
        0.85587911],
       [1.09393837, 1.08088235, 1.02761674, ..., 0.96519561, 0.96173403,
        0.95890284],
       [1.0971893 , 1.07536765, 1.04295431, ..., 1.17025154, 1.18730553,
        1.14242645]])

In [21]:
k,m =5, 5

In [32]:
y = np.asarray(y)
fixed = False
k,m =5, 5
rows, cols = y.shape
ly = ps.lag_spatial(w, y) #spatial lag
if fixed:
    classes_o = mc.Quantiles(y.flatten(), k=k).yb
    classes_o.shape = (rows, cols)
    classes_n = mc.Quantiles(ly.flatten(), k=m).yb
    classes_n.shape = (rows, cols)
else:
    classes_o = np.array([mc.Quantiles(y[:, i], k=k).yb for i
                          in np.arange(cols)]).transpose()
    classes_n = np.array([mc.Quantiles(ly[:, i], k=m).yb for i
                          in np.arange(cols)]).transpose()

markov_o = Markov(classes_o)
markov_n = Markov(classes_n)

joint_null = np.kron(markov_o.p, markov_n.p) #if own-chain and
# neighbor-chain are independent (null)

joint_types = {} #state space {(own,lag),...} and assigned number
c = 0
for i in range(k):
    for j in range(m):
        joint_types[(i,j)] = c
        c += 1

move_types = {} #((own_t,lag_t),(own_t+1,lag_t+1))
for i in joint_types.keys():
    for j in joint_types.keys():
        move_types[(i,j)] = (joint_types[i],joint_types[j])

joint_null = np.array(joint_null)
joint_esti_trans = np.zeros_like(joint_null)
for i in range(rows):
    for j in range(cols-1):
        loc = move_types[((classes_o[i,j],classes_n[i,j]),(classes_o[i,j+1],classes_n[i,j+1]))]
        joint_esti_trans[loc[0],loc[1]] += 1

row_sum = joint_esti_trans.sum(axis=1)
joint_null_trans = np.dot(np.diag(row_sum),joint_null)
dof = k*m*(k*m-1) - k*(k-1) - m*(m-1)

rsnz = row_sum > 0
dof = sum(rsnz)
diff = joint_esti_trans - joint_null_trans
diff2 = np.multiply(diff, diff)
E = joint_null_trans + (joint_null_trans == 0)
chi2 = (diff2 / E).sum()
pvalue = 1 - stats.chi2.cdf(chi2, dof)

In [ ]:
class Joint_Markov(object):
    '''
    Bivariate Markov Chains with (own state, neighbor state) as Markov state.
    
    Notes
    -----
    Could be used to form the tests for contemporaneous spatial dependence 
    in the discrete Markov chains framework: whether the transitional 
    dynamics of each region and the transitional dynamics of its simultaneous 
    local context are dependent. See more in :cite:`Kang2018`.
    
    
    '''
    

In [27]:
np.multiply?

In [28]:
rsnz = row_sum > 0
dof = sum(rsnz)
row_sum = row_sum + (row_sum == 0)
E = joint_null_trans
num = joint_esti_trans - joint_null_trans
num = np.multiply(num, num)
E = E + (E == 0)
chi2 = (num / E).sum()
pvalue = 1 - stats.chi2.cdf(chi2, dof)

In [30]:
dof

25

In [31]:
chi2

1246.8352975144971

In [29]:
pvalue

0.0

In [9]:
def chi2_df(T1, T2, type = "fitness", df=0):
    """
    chi-squared test of difference between two transition matrices.

    Parameters
    ----------
    T1    : matrix
            (k, k), matrix of transitions (counts).
    T2    : matrix
            (k, k), matrix of transitions (counts) to use to form the
            probabilities under the null.

    Returns
    -------
          : tuple
            (3 elements).
            (chi2 value, pvalue, degrees of freedom).

    Examples
    --------

    Notes
    -----
    Second matrix is used to form the probabilities under the null.
    Marginal sums from first matrix are distributed across these probabilities
    under the null. In other words the observed transitions are taken from T1
    while the expected transitions are formed as follows

    .. math::

            E_{i,j} = \sum_j T1_{i,j} * T2_{i,j}/\sum_j T2_{i,j}

    Degrees of freedom corrected for any rows in either T1 or T2 that have
    zero total transitions.
    """
    rs2 = T2.sum(axis=1)
    rs1 = T1.sum(axis=1)
    rs2nz = rs2 > 0
    rs1nz = rs1 > 0
    dof1 = sum(rs1nz)
    dof2 = sum(rs2nz)
    rs2 = rs2 + (rs2 == 0)
    if type == "independence":
        dof = (dof1 - 1) * (dof2 - 1)
        #k = T1.shape[0]**0.5
        #dof = k**2 * (k**2-1) - 2*k*(k-1)
    elif type == "fitness":
        dof = (dof2 - 1) * (dof2 - 0)

    if df != 0:
       dof = df

    p = np.diag(1 / rs2) * np.matrix(T2)
    E = np.diag(rs1) * np.matrix(p)
    num = T1 - E
    num = np.multiply(num, num)
    E = E + (E == 0)
    chi2 = num / E
    chi2 = chi2.sum()
    pvalue = 1 - stats.chi2.cdf(chi2, dof)
    return chi2, pvalue, dof

In [17]:
T2 = joint_null_trans
T1 = joint_esti_trans

In [18]:
rs2 = T2.sum(axis=1)
rs2

array([475.,  98., 175.,  51.,   2., 159., 196., 167., 127.,  73.,  99.,
       211., 176., 230.,  84.,  59., 149., 151., 144., 215.,   8.,  66.,
       131., 168., 426.])

In [19]:
rs1 = T1.sum(axis=1)
rs1

array([475.,  98., 175.,  51.,   2., 159., 196., 167., 127.,  73.,  99.,
       211., 176., 230.,  84.,  59., 149., 151., 144., 215.,   8.,  66.,
       131., 168., 426.])

In [20]:
p = np.diag(1 / rs2) * np.matrix(T2)
p

matrix([[8.73707865e-01, 3.64044944e-02, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 8.50936330e-02, 3.54556804e-03, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 1.19850187e-03, 4.99375780e-05,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00],
        [4.04494382e-02, 7.52106742e-01, 1.16292135e-01, 1.26404494e-03,
         0.00000000e+00, 3.93952004e-03, 7.32504508e-02, 1.13261201e-02,
         1.23110001e-04, 0.00000000e+00, 5.54861978e-05, 1.03169649e-03,
         1.59522819e-04, 1.73394368e-06, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00],
        [0.00000000e+00, 1.03525281e-01, 7.12162921e-01, 9.10112360e-02,

In [10]:
chi2_df(joint_esti_trans, joint_null_trans, df=df)

(1246.8352975144974, 0.0, 560)

In [7]:
giddy.markov.chi2(joint_esti_trans, joint_null_trans)

(1246.8352975144974, 0.0, 576)